In [59]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs 
import requests

In [60]:
url_hoteles = "https://ibis.accor.com/es/destination/city/hotels-madrid-v2418.html"

In [61]:
respuesta_hoteles = requests.get(url_hoteles)
respuesta_hoteles.status_code

200

In [62]:
sopa_hoteles = bs(respuesta_hoteles.content, "html.parser")

In [63]:
hoteles = sopa_hoteles.find_all("li", class_="aem-GridColumn aem-GridColumn--default--3")[:10]
len(hoteles)

10

In [64]:
dictio_scrap = {
    "nombre_hotel": [],
    "descripcion": [],
    "valoracion": []}

In [65]:
for hotel in hoteles:
    # Nombre del hotel (Ejemplo: en un <h2>, <h3> o <a>)
    nombre = hotel.find("h3")
    nombre_hotel = nombre.get_text(strip=True) if nombre else "Sin nombre"

    # Descripción (Ejemplo: en un <p>) 
    descripcion = hotel.find("p")
    descripcion_texto = descripcion.get_text(strip=True) if descripcion else "Sin descripción"

    # Valoración (Ejemplo: en un <span> con una clase específica)
    valoracion = hotel.find("span", class_="cmp-teaser__rating-score")  # Buscar la clase correcta
    valoracion_texto = valoracion.get_text(strip=True) if valoracion else "Sin valoración"
    if valoracion_texto != "Sin valoración":
        valoracion_texto = valoracion_texto.split("/")[0]  # Tomar solo el número antes de "/"
        valoracion_texto = valoracion_texto.replace(",", ".")  # Convertir "4,7" a "4.7"
        valoracion_texto = float(valoracion_texto)  # Convertir a número flotante

    # Agregar los datos al diccionario
    dictio_scrap["nombre_hotel"].append(nombre_hotel)
    dictio_scrap["descripcion"].append(descripcion_texto)
    dictio_scrap["valoracion"].append(valoracion_texto)

In [66]:
df_hoteles = pd.DataFrame(dictio_scrap)
df_hoteles

,nombre_hotel,descripcion,valoracion
0,ibis Styles Madrid Prado,"Estamos en el centro de Madrid, con una ubicac...",4.7
1,ibis budget Madrid Centro Lavapies,Nuestra ubicación te permitirá descubrir por t...,4.3
2,ibis Styles Madrid Centro Maravillas,ibis Styles Madrid Centro Maravillas es un hot...,4.7
3,ibis budget Madrid Calle 30,"Tú, si tú! buscas un hotel ibis budget Madrid ...",4.3
4,ibis Madrid Centro las Ventas,El hotel ibis Madrid Centro las Ventas tiene u...,4.5
5,ibis budget Madrid Centro las Ventas,Ibis budget Madrid Centro está situado a 15 mi...,4.3
6,ibis Styles Madrid City Las Ventas,Junto a la principal arteria madrileña está u...,4.6
7,ibis budget Madrid Calle Alcalá,El ibis budget Madrid Calle Alcalá es el punto...,4.3
8,ibis Madrid Calle Alcalá,"Bienvenido al hotel ibis Madrid Calle Alcalá, ...",4.5
9,ibis budget Madrid Vallecas,ibis budget Madrid Vallecas es un hotel modern...,4.2


In [67]:
df_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre_hotel  10 non-null     object 
 1   descripcion   10 non-null     object 
 2   valoracion    10 non-null     float64
dtypes: float64(1), object(2)
memory usage: 372.0+ bytes


In [68]:
def scrap_info_hoteles(url):
    dictio_scrap = {
        "nombre_hotel": [],
        "id_hotel": [],
        "descripcion": [],
        "valoracion": []
    }

    res_hoteles = requests.get(url)

    if res_hoteles.status_code == 200:
        sopa_hoteles = bs(res_hoteles.content, "html.parser")

        # Buscar los hoteles y manejar caso en que la lista esté vacía
        hoteles = sopa_hoteles.find_all("li", class_="aem-GridColumn aem-GridColumn--default--3")
        hoteles = hoteles[:9] if hoteles else []  # Evita errores de slicing si está vacío

        for hotel in hoteles:
            # Extraer nombre del hotel
            nombre = hotel.find("h3")
            nombre_hotel = nombre.get_text(strip=True) if nombre else "Sin nombre"

            # Extraer descripción
            descripcion = hotel.find("p")
            descripcion_texto = descripcion.get_text(strip=True) if descripcion else "Sin descripción"

            # Extraer valoración
            valoracion = hotel.find("span", class_="cmp-teaser__rating-score")
            valoracion_texto = valoracion.get_text(strip=True) if valoracion else "Sin valoración"

            # Limpiar y convertir la valoración a float
            if valoracion_texto != "Sin valoración":
                valoracion_texto = valoracion_texto.split("/")[0]  # Tomar solo el número antes de "/"
                valoracion_texto = valoracion_texto.replace(",", ".")  # Convertir "4,7" a "4.7"
                try:
                    valoracion_texto = float(valoracion_texto)
                except ValueError:
                    valoracion_texto = "Error en conversión"

            ids_hoteles = []
            # Extraer el ID desde el atributo "data-product-id" para poder sacar el precio_noche de cada hotel.
            div_info = hotel.find("div", class_="cmp-teaser ace-callout-component ace-callout-component-default")
            
            if div_info and "data-product-id" in div_info.attrs:
                id_hotel = div_info["data-product-id"]
            else:
                id_hotel = "Sin ID"

            # Agregar datos al diccionario
            dictio_scrap["nombre_hotel"].append(nombre_hotel)
            dictio_scrap["id_hotel"].append(id_hotel)
            dictio_scrap["descripcion"].append(descripcion_texto)
            dictio_scrap["valoracion"].append(valoracion_texto)

    return dictio_scrap

In [69]:
dictio_final = scrap_info_hoteles(url_hoteles)
df = pd.DataFrame(dictio_final)
df

,nombre_hotel,id_hotel,descripcion,valoracion
0,ibis Styles Madrid Prado,8052,"Estamos en el centro de Madrid, con una ubicac...",4.7
1,ibis budget Madrid Centro Lavapies,A0F2,Nuestra ubicación te permitirá descubrir por t...,4.3
2,ibis Styles Madrid Centro Maravillas,3318,ibis Styles Madrid Centro Maravillas es un hot...,4.7
3,ibis budget Madrid Calle 30,8718,"Tú, si tú! buscas un hotel ibis budget Madrid ...",4.3
4,ibis Madrid Centro las Ventas,7438,El hotel ibis Madrid Centro las Ventas tiene u...,4.5
5,ibis budget Madrid Centro las Ventas,6506,Ibis budget Madrid Centro está situado a 15 mi...,4.3
6,ibis Styles Madrid City Las Ventas,B4U5,Junto a la principal arteria madrileña está u...,4.6
7,ibis budget Madrid Calle Alcalá,5100,El ibis budget Madrid Calle Alcalá es el punto...,4.3
8,ibis Madrid Calle Alcalá,3312,"Bienvenido al hotel ibis Madrid Calle Alcalá, ...",4.5


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre_hotel  9 non-null      object 
 1   id_hotel      9 non-null      object 
 2   descripcion   9 non-null      object 
 3   valoracion    9 non-null      float64
dtypes: float64(1), object(3)
memory usage: 420.0+ bytes


In [71]:
df.to_csv("../data/10_hoteles_competencia.csv", index=False)

In [72]:
# Lista para almacenar los ID de los hoteles
ids_hoteles = []

# Iterar sobre cada hotel y extraer el ID
for hotel in hoteles:
    # Extraer el ID desde el atributo "data-product-id"
    div_info = hotel.find("div", class_="cmp-teaser ace-callout-component ace-callout-component-default")
    
    if div_info and "data-product-id" in div_info.attrs:
        id_hotel = div_info["data-product-id"]
    else:
        id_hotel = "Sin ID"

    ids_hoteles.append(id_hotel)


In [73]:
url_id = "https://all.accor.com/hotel/8052/index.es.shtml"

In [78]:
respuesta_id = requests.get(url_id)
respuesta_id.status_code

200

In [79]:
sopa_id = bs(respuesta_id.content, "html.parser")

In [84]:
precio = sopa_id.find("span", class_="amount")

In [85]:
if precio:
    noche_hotel = precio.get_text(strip=True)

precio


<span class="amount">NaN</span>

In [95]:
def obtener_precio_hotel(codigo_hotel):
    """
    Extrae el precio por noche desde la página individual del hotel.

    Parámetros:
    - codigo_hotel (str): Código único del hotel en la URL.

    Retorna:
    - float o "Sin precio": Precio por noche si se encuentra, o un mensaje de error.
    """
    url_hotel = f"https://all.accor.com/hotel/{codigo_hotel}/index.es.shtml"
    res = requests.get(url_hotel)

    if res.status_code != 200:
        return "Sin precio"

    sopa = bs(res.content, "html.parser")

    # Buscar el precio dentro del <span class="amount">
    precio = sopa.find("span", class_="amount")

    if precio:
        noche_hotel = precio.get_text(strip=True)

        # Convertir el precio a número si se encuentra
        try:
            noche_hotel = float(noche_hotel.replace(",", ".").strip())  # Reemplazar coma por punto para formato numérico
        except ValueError:
            noche_hotel = "Error en conversión"
    else:
        noche_hotel = "Sin precio"

    return noche_hotel

In [96]:
id_hotel = df["id_hotel"][4]
obtener_precio_hotel(id_hotel)

nan

In [97]:
def scrap_info_hoteles(url):
    """
    Extrae información de hoteles desde la página de lista y obtiene el precio por noche de cada uno.

    Parámetros:
    - url (str): URL de la página con la lista de hoteles.

    Retorna:
    - dict: Un diccionario con la información y el precio de cada hotel.
    """
    dictio_scrap = {
        "nombre_hotel": [],
        "id_hotel": [],
        "descripcion": [],
        "valoracion": [],
        "precio_noche": []  # ✅ Se añade la columna de precio
    }

    res_hoteles = requests.get(url)

    if res_hoteles.status_code == 200:
        sopa_hoteles = bs(res_hoteles.content, "html.parser")

        # Buscar los hoteles y manejar caso en que la lista esté vacía
        hoteles = sopa_hoteles.find_all("li", class_="aem-GridColumn aem-GridColumn--default--3")
        hoteles = hoteles[:9] if hoteles else []  # Evita errores de slicing si está vacío

        for hotel in hoteles:
            # Extraer nombre del hotel
            nombre = hotel.find("h3")
            nombre_hotel = nombre.get_text(strip=True) if nombre else "Sin nombre"

            # Extraer descripción
            descripcion = hotel.find("p")
            descripcion_texto = descripcion.get_text(strip=True) if descripcion else "Sin descripción"

            # Extraer valoración
            valoracion = hotel.find("span", class_="cmp-teaser__rating-score")
            valoracion_texto = valoracion.get_text(strip=True) if valoracion else "Sin valoración"

            # Limpiar y convertir la valoración a float
            if valoracion_texto != "Sin valoración":
                valoracion_texto = valoracion_texto.split("/")[0]  # Tomar solo el número antes de "/"
                valoracion_texto = valoracion_texto.replace(",", ".")  # Convertir "4,7" a "4.7"
                try:
                    valoracion_texto = float(valoracion_texto)
                except ValueError:
                    valoracion_texto = "Error en conversión"

            # Extraer el ID del hotel
            div_info = hotel.find("div", class_="cmp-teaser ace-callout-component ace-callout-component-default")
            if div_info and "data-product-id" in div_info.attrs:
                id_hotel = div_info["data-product-id"]
            else:
                id_hotel = "Sin ID"

            # ✅ Obtener el precio del hotel usando el ID
            precio_noche = obtener_precio_hotel(id_hotel) if id_hotel != "Sin ID" else "Sin precio"

            # Agregar datos al diccionario
            dictio_scrap["nombre_hotel"].append(nombre_hotel)
            dictio_scrap["id_hotel"].append(id_hotel)
            dictio_scrap["descripcion"].append(descripcion_texto)
            dictio_scrap["valoracion"].append(valoracion_texto)
            dictio_scrap["precio_noche"].append(precio_noche)  # ✅ Se añade el precio

    return dictio_scrap

In [98]:
dictio_final = scrap_info_hoteles(url_hoteles)
df = pd.DataFrame(dictio_final)
df

,nombre_hotel,id_hotel,descripcion,valoracion,precio_noche
0,ibis Styles Madrid Prado,8052,"Estamos en el centro de Madrid, con una ubicac...",4.7,NaN
1,ibis budget Madrid Centro Lavapies,A0F2,Nuestra ubicación te permitirá descubrir por t...,4.3,NaN
2,ibis Styles Madrid Centro Maravillas,3318,ibis Styles Madrid Centro Maravillas es un hot...,4.7,NaN
3,ibis budget Madrid Calle 30,8718,"Tú, si tú! buscas un hotel ibis budget Madrid ...",4.3,NaN
4,ibis Madrid Centro las Ventas,7438,El hotel ibis Madrid Centro las Ventas tiene u...,4.5,NaN
5,ibis budget Madrid Centro las Ventas,6506,Ibis budget Madrid Centro está situado a 15 mi...,4.3,NaN
6,ibis Styles Madrid City Las Ventas,B4U5,Junto a la principal arteria madrileña está u...,4.6,NaN
7,ibis budget Madrid Calle Alcalá,5100,El ibis budget Madrid Calle Alcalá es el punto...,4.3,NaN
8,ibis Madrid Calle Alcalá,3312,"Bienvenido al hotel ibis Madrid Calle Alcalá, ...",4.5,NaN
